In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import json
from PIL import Image  
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn.functional as F
from monai.transforms import (
    Compose, RandFlip, RandRotate, RandAffine, Resize
)
import torch.nn as nn
from fastai.learner import Learner
from monai.networks.nets import resnet18
from fastai.data.core import DataLoaders
from fastai.metrics import accuracy
from fastai.losses import CrossEntropyLossFlat
from fastai.callback.all import SaveModelCallback, EarlyStoppingCallback
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
%matplotlib inline

In [ ]:
# # Load datasets
# df = pd.read_csv('Final_Datasets/blood_flow_heart.csv')
# df_test = pd.read_csv('Final_Datasets/test_data_incidence.csv')
# controls = pd.read_csv('Final_Datasets/imbalanced_control_iid.csv')

# # Remove test cases and controls
# test_iids = set(df_test['IID'])

# # Get all cases (CAD = 1) from blood_flow_heart.csv, excluding test data
# cases_train = df[(df['CAD'] == 1) & (~df['IID'].isin(test_iids))]

# # Get all controls (IID in controls.csv and CAD = 0), excluding test data
# controls_train = df[(df['IID'].isin(controls['IID'])) & (df['CAD'] == 0) & (~df['IID'].isin(test_iids))]

# # Combine the imbalanced dataset
# df_imbalanced_train = pd.concat([cases_train, controls_train]).reset_index(drop=True)

# # Save the imbalanced dataset
# df_imbalanced_train.to_csv('Final_Datasets/train_imbalanced_blood_flow.csv', sep=',', index=False)

# # Report the dataset statistics
# cases_count = len(cases_train)
# controls_count = len(controls_train)
# print(f"Number of cases: {cases_count}")
# print(f"Number of controls: {controls_count}")
# print(f"Imbalance ratio (controls to cases): {controls_count / cases_count:.2f}")

In [ ]:
# class NpyDataset(Dataset):
#     def __init__(self, dataframe, image_column_name, label_column_name):
#         self.dataframe = dataframe.reset_index(drop=True)
#         self.image_column_name = image_column_name
#         self.label_column_name = label_column_name
#         self.transform = transforms.Compose([
#             transforms.Lambda(lambda x: torch.stack([x, x, x], dim=0) if x.size(0) == 18 else x),  # Stack to create 3 channels
#             transforms.Resize((112, 112))  # Resize to 112x112
#         ])

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, idx):
#         npy_path = self.dataframe.loc[idx, self.image_column_name]
#         label = self.dataframe.loc[idx, self.label_column_name]
#         image = np.load(npy_path)  # Load NPY file (e.g., shape: (30, 200, 200))

#         # Access channel and frames specific to blood flow data 
#         image = image[6:24, :, :]  # Selecting frames 6 to 23 (to get 18 frames)

#         image = torch.tensor(image, dtype=torch.float32)  # Convert to tensor
#         image = self.transform(image)  # Apply transforms
#         label = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
#         return {'image': image, 'label': label, 'path': npy_path}
    


# class HeartDiseaseModelBloodFlow:
#     def __init__(self, train_df_path, test_df_path, image_column_name, label_column_name, batch_size=32, model_name='blood_flow_model'):
#         self.train_df_path = train_df_path
#         self.test_df_path = test_df_path
#         self.image_column_name = image_column_name
#         self.label_column_name = label_column_name
#         self.batch_size = batch_size
#         self.model_name = model_name
#         self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
#         self._prepare_data()
#         self._prepare_model()

#     def _prepare_data(self):
#         train_df = pd.read_csv(self.train_df_path)
#         test_df = pd.read_csv(self.test_df_path)

#         self.train_dataset = NpyDataset(train_df, self.image_column_name, self.label_column_name)
#         self.test_dataset = NpyDataset(test_df, self.image_column_name, self.label_column_name)

#         self.train_loader = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=8)
#         self.test_loader = DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=8)

#     def _prepare_model(self):
#         self.model = models.video.r3d_18(pretrained=True)
#         self.model.fc = nn.Linear(self.model.fc.in_features, 2)
#         self.model.to(self.device)

#         self.learn = Learner(
#             DataLoaders(self.train_loader, self.test_loader),
#             self.model,
#             loss_func=CrossEntropyLossFlat(),
#             metrics=[accuracy],
#             wd=1e-4,
#             cbs=[
#                 SaveModelCallback(fname=self.model_name, monitor='train_loss'),
#                 EarlyStoppingCallback(monitor='train_loss', patience=10)
#             ]
#         ).to_fp16()

#     def train(self, epochs=5, lr=1e-5):
#         self.learn.fine_tune(epochs, base_lr=lr)

#     def extract_embeddings(self, loader):
#         self.model.eval()
#         embeddings, labels, paths = [], [], []
#         with torch.no_grad():
#             for batch in loader:
#                 images = batch['image']
#                 label_batch = batch['label']
#                 path_batch = batch['path']
#                 images = images.to(self.device)
                
#                 # Pass through the model until the final pooling layer
#                 x = self.model.stem(images)
#                 x = self.model.layer1(x)
#                 x = self.model.layer2(x)
#                 x = self.model.layer3(x)
#                 x = self.model.layer4(x)
#                 x = self.model.avgpool(x)
#                 x = torch.flatten(x, 1)
                
#                 embeddings.append(x.cpu().numpy())
#                 labels.append(label_batch.cpu().numpy())
#                 paths.extend(path_batch)  # Collecting the paths
#         embeddings = np.concatenate(embeddings)
#         labels = np.concatenate(labels)
#         return embeddings, labels, paths

#     # Updated generate_embeddings_dataframe function
#     def generate_embeddings_dataframe(self, embeddings, labels, paths):
#         """
#         Creates a Pandas DataFrame from embeddings, labels, and image paths.

#         Args:
#             embeddings (numpy.ndarray): The extracted embeddings.
#             labels (numpy.ndarray): The labels corresponding to the embeddings.
#             paths (list of str): The image paths.

#         Returns:
#             pd.DataFrame: A DataFrame with serialized embeddings and metadata.
#         """
#         # Serialize embeddings as JSON strings for safe CSV storage
#         df = pd.DataFrame({
#             'image_path': paths,
#             'embedding': [json.dumps(emb.tolist()) for emb in embeddings],
#             'label': labels
#         })
#         return df

#     # Updated extract_and_save_embeddings function
#     def extract_and_save_embeddings(self):
#         """
#         Extracts embeddings for train and test datasets and saves them as CSV files.

#         The embeddings are serialized as JSON strings for robust CSV storage.
#         """
#         # Extract training embeddings
#         train_embeddings, train_labels, train_paths = self.extract_embeddings(self.train_loader)
#         train_df = self.generate_embeddings_dataframe(train_embeddings, train_labels, train_paths)

#         # Extract test embeddings
#         test_embeddings, test_labels, test_paths = self.extract_embeddings(self.test_loader)
#         test_df = self.generate_embeddings_dataframe(test_embeddings, test_labels, test_paths)
        
#         # Save DataFrames
#         train_df.to_csv('train_embeddings_blood_flow_nov_imbalanced.csv', index=False)
#         test_df.to_csv('test_embeddings_blood_flow_nov.csv', index=False)

In [ ]:
# if __name__ == "__main__":
#     model = HeartDiseaseModelBloodFlow(
#         train_df_path='Final_Datasets/train_imbalanced_blood_flow.csv',  # Updated for blood flow dataset
#         test_df_path='Final_Datasets/test_data_incidence.csv',      # Updated for test data
#         image_column_name='FilePath_bf',  # Replace with blood flow image path column name
#         label_column_name='CAD',          # Replace with blood flow label column name
#         model_name='heart_blood_flow'
#     )
#     # Uncomment the following line if you need to train the model first
#     # model.train(epochs=50, lr=1e-5)
    
#     # Extract embeddings and save to CSV
#     model.extract_and_save_embeddings()

In [ ]:
# emb = pd.read_csv('train_embeddings_blood_flow_nov_imbalanced.csv')
# emb['embedding'] = emb['embedding'].apply(lambda x: np.array(json.loads(x)))
# emb.head()

In [ ]:
# len(emb['embedding'][0])

**MONAI RESNET**

In [ ]:
# class NPYDataset(Dataset):
#     """
#     Custom PyTorch Dataset for loading 3D medical imaging data from .npy files.
#     """
#     def __init__(self, dataframe, image_column_name, label_column_name):
#         """
#         Args:
#             dataframe (pd.DataFrame): The dataframe containing the data.
#             image_column_name (str): Name of the column containing the image file paths.
#             label_column_name (str): Name of the column containing the labels.
#         """
#         self.dataframe = dataframe.reset_index(drop=True)
#         self.image_column_name = image_column_name
#         self.label_column_name = label_column_name

#         # Transformation pipeline
#         self.transform = transforms.Compose([
#             transforms.Resize((112, 112))  # Resize to 112x112
#         ])

#     def __len__(self):
#         """Return the number of samples in the dataset."""
#         return len(self.dataframe)

#     def __getitem__(self, idx):
#         """
#         Load and preprocess a single sample.

#         Args:
#             idx (int): Index of the sample to fetch.

#         Returns:
#             dict: Contains the processed image, label, and original file path.
#         """
#         try:
#             npy_path = self.dataframe.loc[idx, self.image_column_name]
#             label = self.dataframe.loc[idx, self.label_column_name]

#             # Load the 3D array (e.g., shape: [30, 200, 200])
#             image = np.load(npy_path)

#             # Select exactly 18 frames for the model
#             image = image[6:24, :, :]  # Shape: [18, 200, 200]

#             # Convert to tensor and add channel dimension
#             image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Shape: [1, 18, 200, 200]

#             # Resize spatial dimensions (height and width) to 112x112
#             image = F.interpolate(image, size=(112, 112), mode='bilinear', align_corners=False)

#             # Apply transformations
#             image = self.transform(image)

#             label = torch.tensor(label, dtype=torch.long)  # Convert label to tensor

#             return {'image': image, 'label': label, 'path': npy_path}

#         except Exception as e:
#             raise RuntimeError(f"Error processing file at index {idx}: {e}")
            
import os
import numpy as np
import pandas as pd
import torch
import json
from PIL import Image  
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn.functional as F
from monai.transforms import (
    Compose, Resize
)
import torch.nn as nn
from fastai.learner import Learner
from monai.networks.nets import resnet18
from fastai.data.core import DataLoaders
from fastai.metrics import accuracy
from fastai.losses import CrossEntropyLossFlat
from fastai.callback.all import SaveModelCallback, EarlyStoppingCallback

class NPYDataset(Dataset):
    """
    Custom PyTorch Dataset for loading 3D medical imaging data from .npy files.
    """
    def __init__(self, dataframe, image_column_name, label_column_name):
        self.dataframe = dataframe
        self.image_column_name = image_column_name
        self.label_column_name = label_column_name

        # Transformation pipeline
        self.transform = Compose([
            Resize(spatial_size=(18, 112, 112))
        ])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        """
        Load and preprocess a single sample.
        """
        try:
            npy_path = self.dataframe[self.image_column_name].iloc[idx]
            label = self.dataframe[self.label_column_name].iloc[idx]

            # Load the 3D array (100 frames, 224x224 pixels)
            image = np.load(npy_path)  # Shape: (100, 224, 224)

            # Select exactly 18 frames
            image = image[45:63, :, :]  # Shape: (18, 224, 224)

            # Convert to tensor and add channel dimension
            image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Shape: (1, 18, 224, 224)

            # Resize spatial dimensions to 112x112
            image = F.interpolate(image, size=(112, 112), mode='bilinear', align_corners=False)

            # Apply transformations
            image = self.transform(image)

            return {'image': image, 'label': label, 'path': npy_path}

        except Exception as e:
            print(f"Error loading image at index {idx}: {e}")
            raise

class HeartDiseaseModelBloodFlow:
    def __init__(self, train_df_path, test_df_path, image_column_name, label_column_name, batch_size=32, model_name='heart_ch0_3channel_MONAI_resnet18'):
        self.train_df_path = train_df_path
        self.test_df_path = test_df_path
        self.image_column_name = image_column_name
        self.label_column_name = label_column_name
        self.batch_size = batch_size
        self.model_name = model_name
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self._prepare_data()
        self._load_model()

    def _prepare_data(self):
        train_df = pd.read_csv(self.train_df_path)
        test_df = pd.read_csv(self.test_df_path)

        self.train_dataset = NPYDataset(train_df, self.image_column_name, self.label_column_name)
        self.test_dataset = NPYDataset(test_df, self.image_column_name, self.label_column_name)

        self.train_loader = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=8)
        self.test_loader = DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=8)

    def _load_model(self):
        # Create an instance of MONAI's ResNet18
        self.model = resnet18(spatial_dims=3, n_input_channels=1, num_classes=2)

        # Initialize FastAI Learner
        self.learn = Learner(
            DataLoaders(self.train_loader, self.test_loader),
            self.model,
            loss_func=CrossEntropyLossFlat(),
            metrics=[accuracy]
        ).to_fp16()

        # Load the fine-tuned model
        self.learn.load(self.model_name)
        self.model = self.learn.model.eval().to(self.device)

    ### Extract Embeddings (Separate)
    def extract_embeddings(self, loader):
        embeddings, labels, paths = [], [], []
        with torch.no_grad():
            for batch in loader:
                images = batch['image'].to(self.device)
                label_batch = batch['label']
                path_batch = batch['path']
                
                # Extract embeddings from the penultimate layer
                x = self.model.conv1(images)
                x = self.model.bn1(x)
                x = self.model.maxpool(x)
                x = self.model.layer1(x)
                x = self.model.layer2(x)
                x = self.model.layer3(x)
                x = self.model.layer4(x)
                x = self.model.avgpool(x)
                x = torch.flatten(x, 1)

                embeddings.append(x.cpu().numpy())
                labels.append(label_batch.cpu().numpy())
                paths.extend(path_batch)  
                
        embeddings = np.concatenate(embeddings)
        labels = np.concatenate(labels)
        return embeddings, labels, paths

    def generate_embeddings_dataframe(self, embeddings, labels, paths):
        df = pd.DataFrame({
            'image_path': paths,
            'embedding': [json.dumps(emb.tolist()) for emb in embeddings],
            'label': labels
        })
        return df

    def extract_and_save_embeddings(self):
        train_embeddings, train_labels, train_paths = self.extract_embeddings(self.train_loader)
        train_df = self.generate_embeddings_dataframe(train_embeddings, train_labels, train_paths)

        test_embeddings, test_labels, test_paths = self.extract_embeddings(self.test_loader)
        test_df = self.generate_embeddings_dataframe(test_embeddings, test_labels, test_paths)

        train_df.to_csv('train_embeddings_aortic_monai.csv', index=False)
        test_df.to_csv('test_embeddings_aortic_monai.csv', index=False)

    ### Extract Predicted Probabilities (Separate)
    def extract_probabilities(self, loader):
        probabilities, labels, paths = [], [], []
        with torch.no_grad():
            for batch in loader:
                images = batch['image'].to(self.device)
                label_batch = batch['label']
                path_batch = batch['path']

                logits = self.model(images)  # Direct model output
                probs = F.softmax(logits, dim=1)[:, 1]  # Probability of CAD = 1

                probabilities.append(probs.cpu().numpy())
                labels.append(label_batch.cpu().numpy())
                paths.extend(path_batch)
            
        probabilities = np.concatenate(probabilities)
        labels = np.concatenate(labels)
        return probabilities, labels, paths

    def generate_probabilities_dataframe(self, probabilities, labels, paths):
        df = pd.DataFrame({
            'image_path': paths,
            'predicted_probability': probabilities,
            'label': labels
        })
        return df

    def extract_and_save_probabilities(self):
        train_probs, train_labels, train_paths = self.extract_probabilities(self.train_loader)
        train_df = self.generate_probabilities_dataframe(train_probs, train_labels, train_paths)

        test_probs, test_labels, test_paths = self.extract_probabilities(self.test_loader)
        test_df = self.generate_probabilities_dataframe(test_probs, test_labels, test_paths)

        train_df.to_csv('train_aortic_predicted_probabilities.csv', index=False)
        test_df.to_csv('test_aortic_predicted_probabilities.csv', index=False)

In [ ]:
if __name__ == "__main__":
    model = HeartDiseaseModelBloodFlow(
        train_df_path='Final_Datasets/train_resnet_ao.csv',
        test_df_path='Final_Datasets/test_incidence_new.csv',
        image_column_name='FilePath_ao',
        label_column_name='CAD',
        model_name='heart_ao_Med3D_resnet18'
    )
    
    model.extract_and_save_embeddings()  # Save embeddings
    model.extract_and_save_probabilities()  # Save predicted probabilities

In [ ]:
train = pd.read_csv('train_embeddings_aortic_monai.csv')
train['embedding'] = train['embedding'].apply(lambda x: np.array(json.loads(x)))
train.head()

In [ ]:
# Convert label to categorical (optional for better plotting)
train['label'] = train['label'].astype(str)  # Ensure labels are treated as categorical

# Create a box plot
plt.figure(figsize=(8, 6))
sns.boxplot(x=train['label'], y=train['predicted_probability'], palette="Set2")

# Add labels and title
plt.xlabel("True Label (CAD)", fontsize=12)
plt.ylabel("Predicted Probability (P(CAD=1))", fontsize=12)
plt.title("Predicted Probabilities by True Label", fontsize=14)

# Show the plot

In [ ]:
test = pd.read_csv('test_embeddings_aortic_monai.csv')
test['embedding'] = test['embedding'].apply(lambda x: np.array(json.loads(x)))
test.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt

# Assuming train and test dataframes are named 'train' and 'test'
# Expand embeddings into separate columns for train and test
embedding_columns = [f'embedding_{i}' for i in range(len(train['embedding'][0]))]
train_embedding_df = pd.DataFrame(train['embedding'].tolist(), columns=embedding_columns)
train_data = pd.concat([train_embedding_df, train['label']], axis=1)

test_embedding_df = pd.DataFrame(test['embedding'].tolist(), columns=embedding_columns)
test_data = pd.concat([test_embedding_df, test['label']], axis=1)

# Balance the training data
cases = train_data[train_data['label'] == 1]
controls = train_data[train_data['label'] == 0]
min_size = min(len(cases), len(controls))
cases_balanced = cases.sample(n=min_size, random_state=42)
controls_balanced = controls.sample(n=min_size, random_state=42)
balanced_train_data = pd.concat([cases_balanced, controls_balanced])

# Define features and target
X_train = balanced_train_data[embedding_columns]
y_train = balanced_train_data['label']
X_test = test_data[embedding_columns]
y_test = test_data['label']

# Apply PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Number of PCs to use
pcs_to_use = [5, 10, 15, 20, 25, len(embedding_columns)]
# Dictionary to store ROC data and accuracy
roc_data = {}

# Train models and evaluate
for num_pcs in pcs_to_use:
    if num_pcs == len(embedding_columns):
        X_train_selected = X_train
        X_test_selected = X_test
    else:
        X_train_selected = X_train_pca[:, :num_pcs]
        X_test_selected = X_test_pca[:, :num_pcs]
    
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    y_pred_proba = model.predict_proba(X_test_selected)[:, 1]
    
    # Calculate metrics
    auc = roc_auc_score(y_test, y_pred_proba)
    accuracy = accuracy_score(y_test, y_pred)
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    
    # Store data
    roc_data[num_pcs] = (fpr, tpr, auc, accuracy)

# Plot the ROC curves
plt.figure(figsize=(10, 8))
for num_pcs, (fpr, tpr, auc, accuracy) in roc_data.items():
    label = (
        f"{num_pcs} PCs (AUC = {auc:.2f}, Acc = {accuracy:.2f})"
        if num_pcs != len(embedding_columns)
        else f"All Embeddings (AUC = {auc:.2f}, Acc = {accuracy:.2f})"
    )
    plt.plot(fpr, tpr, label=label)

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.title("ROC Curves for Different Number of PCs for Aortic Distensibilty")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

# Load data
train = pd.read_csv("train_embeddings_aortic_monai.csv")
test = pd.read_csv("test_embeddings_aortic_monai.csv")

# Convert JSON-encoded embeddings into NumPy arrays
train["embedding"] = train["embedding"].apply(lambda x: np.array(json.loads(x)))
test["embedding"] = test["embedding"].apply(lambda x: np.array(json.loads(x)))

# Extract embeddings and labels
X_train = np.vstack(train["embedding"].values)
X_test = np.vstack(test["embedding"].values)
y_train = train["label"].values
y_test = test["label"].values

# Apply PCA to embeddings
pca = PCA(n_components=25)  # Keep 25 PCs
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Number of principal components to evaluate (including full embedding case)
pcs_to_use = [5, 10, 15, 20, 25]  # Already PCA transformed
pcs_to_use.append("Full Embedding")  # Full embeddings without PCA reduction

# Cross-validation setup
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store results
cv_results = []
test_results = []

# Train models and evaluate
for num_pcs in pcs_to_use:
    if num_pcs == "Full Embedding":
        X_train_selected = X_train  # Use full raw embeddings
        X_test_selected = X_test
    else:
        X_train_selected = X_train_pca[:, :num_pcs]
        X_test_selected = X_test_pca[:, :num_pcs]

    # Store all cross-validation performance for box plots
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_selected, y_train)):
        X_train_fold, X_val_fold = X_train_selected[train_idx], X_train_selected[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Train XGBoost classifier
        model = XGBClassifier(eval_metric="logloss")
        model.fit(X_train_fold, y_train_fold)

        # Predict and evaluate on validation set
        y_val_pred_proba = model.predict_proba(X_val_fold)[:, 1]
        auc_val = roc_auc_score(y_val_fold, y_val_pred_proba)
        acc_val = accuracy_score(y_val_fold, (y_val_pred_proba > 0.5).astype(int))

        # Store all CV results
        cv_results.append({
            "Num_PCs": num_pcs,
            "Fold": fold + 1,
            "AUC": auc_val,
            "Accuracy": acc_val
        })

    # Train on full training set and evaluate on test set
    model.fit(X_train_selected, y_train)
    y_test_pred_proba = model.predict_proba(X_test_selected)[:, 1]

    auc_test = roc_auc_score(y_test, y_test_pred_proba)
    acc_test = accuracy_score(y_test, (y_test_pred_proba > 0.5).astype(int))

    # Store test results
    test_results.append({
        "Num_PCs": num_pcs,
        "AUC": auc_test,
        "Accuracy": acc_test
    })

# Convert results to DataFrames and save as CSV
cv_results_df = pd.DataFrame(cv_results)
test_results_df = pd.DataFrame(test_results)

cv_results_csv_path = "pca_xgboost_cv_results.csv"
test_results_csv_path = "pca_xgboost_test_results.csv"

cv_results_df.to_csv(cv_results_csv_path, index=False)
test_results_df.to_csv(test_results_csv_path, index=False)

print(f"Cross-validation results saved to {cv_results_csv_path}")
print(f"Test results saved to {test_results_csv_path}")


In [14]:
len(train)

30749